In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thr Apr 9 16:31:22 2020
@author: Yanzhe
1) Get all needed data from MongoDB and Git
2) Data preparation for analysis
3) Lag effect analysis
4) Random Forest Modeling

"""
#pip install pymongo

In [ ]:
# Import required packages
import pandas as pd
import numpy as np
import pymongo
import warnings
warnings.filterwarnings("ignore")

# Create lagged mobility data as variables
def lag_var(input_df,range0,range1):
    # Slice and Index
    df = input_df[['date','State','m50_percent_of_normal']]
    df = df.sort_values(['State','date'],ascending = (True,True))
    df = df.set_index(["State", "date"]) 

    # Index the input df
    input_df = input_df.set_index(["State", "date"]) 
    input_df.reset_index(drop=False, inplace=True)
        
    # Shift with lag step = (range0 , range1)
    for i in range(range0,range1+1):
        df_new = df
        df_new = df_new.shift(i)
        
        # Reset index
        df_new.reset_index(drop=False, inplace=True)
        col_title = 'LagDay_'+str(i)
        df_new = df_new.rename(columns={'m50_percent_of_normal':col_title})
        # Add to the input df
        input_df[col_title] = df_new[col_title]
    input_df.fillna(100, inplace=True)
    #print(input_df.head()
    
    return input_df

In [ ]:
# Visual by state/county
def pdf_plots(output_df):
    df = output_df
    
    # Get list of states/counties
    states = df['State'].unique()
    print(states)
    j=0
    for state in states:
        dfplot = df.loc[df['State'] == state]
        counties = df['County'].unique()
        for county in counties:
            dfplot = df.loc[df['County'] == county]
            dfplot.sort_values(['date'],ascending = (True))
            # Create a plot for each different lag
            for i in range(0,15):
                fig, ax1 = plt.subplots()
                column_i = 'LagDay_'+str(i)
                x = dfplot['date']
                y1 = dfplot[column_i]
                y2 = dfplot['derive']

                color = 'tab:red'
                ax1.set_xlabel('date')
                ax1.set_ylabel('mobility', color=color)
                ax1.plot(x, y1, color=color)
                ax1.tick_params(axis='y', labelcolor=color)

                # instantiate a second axes that shares the same x-axis
                ax2 = ax1.twinx()
                ax2.set_ylim(0,0.4)
                color = 'tab:blue'
                ax2.set_ylabel('derive', color=color)
                # already handled the x-label with ax1
            
                ax2.plot(x, y2, color=color)
                ax2.tick_params(axis='y', labelcolor=color)
                plt.title(state+' '+column_i)
                fig.tight_layout()
            
                # Save to PDF
                pdf.savefig(fig)
                plt.close()
                #plt.show()
    pdf.close()